In [1]:
from random import choice, sample, randrange, random, shuffle
from string import ascii_lowercase
import numpy as np

### Defining the fitness function

$fitness(w) = \sum^{nºguesses}_{i=1}(|correct(g_i,w) - c_i|+|misplaced(g_i,w) - m_i|)$

Where $w$ is a word for which we will evaluate the fitness, $g_i$ is the guess played on turn $i$, $c_i$ is the number of correct values obtained in the guess $i$, $correct(g,r)$ return the number of correct positions in a Wordle game provided that $g$ is the attempt and $w$ is the secret word, $misplaced(g,w)$ returns the number of misplaced letters given that $g$ is the attempt and $w$ is the secret word, $m_i$ is the number of misplaced letters obtained in the attempt $i$. 

In [2]:
def find(s, ch):
    return [i for i, ltr in enumerate(s) if ltr == ch]

def check_words(input_word ,target):
    input_word = input_word.lower()
    target = target.lower()
    sequence = ['⬜']*5
    selected = [False]*5
    for i in range(0,5):
        if input_word[i] == target[i]:
            sequence[i] = '🟩'
            selected[i] = True
    for i in range(0,5):
        if input_word[i] != target[i]:
            indexes = find(target, input_word[i])
            for index in indexes:
                if not selected[index]:
                    selected[index] = True
                    sequence[i] = '🟨'
                    break
    aux = ""
    return aux.join(sequence)

In [9]:
game_state = [('Hello', '🟩🟩🟨🟨🟨')]

In [245]:
def fitness(word, game_state):
    sum = 0
    for guess, result in game_state:
        match = result.count('🟩')
        misplaced =  result.count('🟨')
        result = check_words(guess, word)
        pos_match = result.count('🟩')
        pos_misplaced = result.count('🟨')
        sum += abs(pos_match - match) + abs(pos_misplaced - misplaced)
    return sum + 2*5*(len(game_state)-1)

### Defining mutation
Mutation occurs when a random letter of the word is changed.

In [4]:
def mutate(word, prob=0.03):
    if random() < prob:
        word = list(word)
        word[randrange(len(word))] = choice(ascii_lowercase)
        word = ''.join(word)
    return word

### Defining permutation
Permutation occurs when two letters from a word are switched

In [5]:
def permute(word, prob=0.03):
    if random() < prob:
        idx1, idx2 = sample(range(len(word)), 2)
        word = list(word)
        word[idx1], word[idx2] = word[idx2], word[idx1]
        word = ''.join(word)
    return word

### Defining inversion
Inversion occurs when two positions are randomly selected and the sequence of letters between is inverted.

In [6]:
def invert(word, prob=0.02):
    if random() < prob:
        idx1, idx2 = sample(range(len(word)), 2)
        idx1, idx2 = min(idx1,idx2), max(idx1,idx2)
        word = list(word)
        word[idx1:idx2] = reversed(word[idx1:idx2])
        word = ''.join(word)
    return word

### Defining crossover

In [7]:
def crossover(parent1, parent2, prob=0.5):
    child1, child2 = parent1, parent2
    if random() < prob:
        split = randrange(1, len(parent1)-1)
        child1 = parent1[:split] + parent2[split:]
        child2 = parent2[:split] + parent1[split:]
    return [child1, child2]

### Defining selection
We will use tournament based selection.

In [13]:
def selection(fitnesses, population, k=20, keep_size=True):
    k = min(k, len(population))
    def random_tournament():
        selected_idx = randrange(len(population))
        for idx in sample(range(len(population)), k=k):
            if fitnesses[idx] > fitnesses[selected_idx]:
                selected_idx = idx
        return population[selected_idx]
    return [random_tournament() for _ in range(len(population))]

### Creating the model

In [49]:
def wordle_genetic(game_state, guess_count, pop_size, possible_guesses, max_gen, tour_size, crossover_prob,
                   mutate_prob, permute_prob, invert_prob):
    
    if guess_count == 0:
        possible_guesses.remove('about')
        return "ABOUT"

    # create population
    sample_size = min(pop_size, len(possible_guesses))
    population = sample(possible_guesses, sample_size)
    eligible_words = set()
    generation = 0

    # do genetic iterations
    while generation < max_gen:
        # selection
        fitnesses = [fitness(p, game_state) for p in population]
        selected = selection(fitnesses, population, k=tour_size)

        # new generation
        new_pop = []
        for p1, p2 in zip(selected[0::2], selected[1::2]):
            for c in crossover(p1, p2, prob=crossover_prob):
                c = mutate(c, prob=mutate_prob)
                c = permute(c, prob=permute_prob)
                c = invert(c, prob=invert_prob)

                if (c in eligible_words) or (c not in possible_guesses):
                    new_pop.append(choice(tuple(possible_guesses)))
                else:
                    new_pop.append(c)

        population = new_pop
        eligible_words.update(population)

        generation += 1

    # choose word in eligible_words with maximum
    best_word = min(eligible_words, key=lambda x: fitness(x, game_state))
    possible_guesses.remove(best_word)
    return best_word

In [33]:
valid_inputs = []
with open("valid-wordle-words.txt") as f:
    valid_inputs = f.read().splitlines()

In [75]:
target = 'abbey'
eligible_words = set(valid_inputs)
game_state = []
word = wordle_genetic(game_state = game_state, guess_count = 0, pop_size = 150, 
                                     possible_guesses = eligible_words, max_gen = 100, tour_size = 40,
                                    crossover_prob = 0.5,  mutate_prob = 0.03, permute_prob = 0.03, invert_prob = 0.02)
check_words(word, target)
game_state.append((word, check_words(word, target)))
print(str(game_state[-1][1]) + " " +game_state[-1][0])
i = 1
while game_state[-1][1] != "🟩🟩🟩🟩🟩" and i <= 5:
    word = wordle_genetic(game_state = game_state, guess_count = i, pop_size = 150, 
                                     possible_guesses = eligible_words, max_gen = 100, tour_size = 40,
                                    crossover_prob = 0.5,  mutate_prob = 0.03, permute_prob = 0.03, invert_prob = 0.02)
    check_words(word, target)
    game_state.append((word, check_words(word, target)))
    print(str(game_state[-1][1]) + " " + game_state[-1][0])
    i += 1
if game_state[-1][1] == "🟩🟩🟩🟩🟩":
    print("Ended with a win in " + str(i) + " attempts")
else:
    print('Ended with a loss')

🟩🟩⬜⬜⬜ ABOUT


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_14508\3451594227.py:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  population = sample(possible_guesses, sample_size)


🟩🟩⬜⬜⬜ abaca
🟩🟩⬜🟩⬜ ables
🟩🟩🟨⬜⬜ abyss
🟩🟩🟩🟩⬜ abbed
🟩🟩🟩🟩🟩 abbey
Ended with a win in 6 attempts


In [76]:
def tryout(target):
    eligible_words = set(valid_inputs)
    game_state = []
    word = wordle_genetic(game_state = game_state, guess_count = 0, pop_size = 150, 
                                         possible_guesses = eligible_words, max_gen = 100, tour_size = 40,
                                        crossover_prob = 0.5,  mutate_prob = 0.03, permute_prob = 0.03, invert_prob = 0.02)
    check_words(word, target)
    game_state.append((word, check_words(word, target)))
    print(str(game_state[-1][1]) + " " +game_state[-1][0])
    i = 1
    while game_state[-1][1] != "🟩🟩🟩🟩🟩" and i <= 5:
        word = wordle_genetic(game_state = game_state, guess_count = i, pop_size = 150, 
                                         possible_guesses = eligible_words, max_gen = 100, tour_size = 40,
                                        crossover_prob = 0.5,  mutate_prob = 0.03, permute_prob = 0.03, invert_prob = 0.02)
        check_words(word, target)
        game_state.append((word, check_words(word, target)))
        print(str(game_state[-1][1]) + " " + game_state[-1][0])
        i += 1
    if game_state[-1][1] == "🟩🟩🟩🟩🟩":
        print("Ended with a win in " + str(i) + " attempts")
    else:
        print("Ended with a loss")

In [77]:
tryout('hello')

⬜⬜🟨⬜⬜ ABOUT


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_14508\3451594227.py:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  population = sample(possible_guesses, sample_size)


⬜⬜⬜🟨⬜ payer
⬜⬜⬜🟩🟨 brill
⬜🟩⬜⬜⬜ bewig
⬜🟩🟩🟩🟩 cello
⬜🟩🟩🟩🟩 jello
Ended with a loss


In [82]:
tryout('where')

⬜⬜⬜⬜⬜ ABOUT


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_14508\3451594227.py:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  population = sample(possible_guesses, sample_size)


⬜⬜⬜⬜⬜ dingy
⬜🟨🟩⬜⬜ seeks
⬜⬜🟩⬜🟩 fleme
⬜🟨🟩🟨🟩 crewe
🟩🟩🟩🟩🟩 where
Ended with a win in 6 attempts


### Modifications using the data
#### Choosing the first attempt

In [146]:
frecs = np.load('new_word_frequency_per_cluster.npy', allow_pickle = True)
frec0 = frecs[0]
g0_sorted = sorted(frec0.items(), key=lambda x:x[1], reverse = True)
g0_top = g0_sorted[:30]
g0_top = np.asarray(g0_top)

In [147]:
top_vals = g0_top[:,1].astype(float)
top_vals = top_vals/top_vals.sum()
top_labels = g0_top[:,0]

In [148]:
def first_word(labels, vals):
    return labels[np.random.choice(len(labels), p = vals)]

#### Guesses based on freq

In [157]:
def wordle_genetic(game_state, guess_count, pop_size, dict_frec, max_gen, tour_size, crossover_prob,
                   mutate_prob, permute_prob, invert_prob, top_vals, top_labels):
    
    if guess_count == 0:
        guess = first_word(top_labels, top_vals)
        dict_frec.pop(guess)
        return guess
    
    possible_guesses = np.asarray(list(dict_frec.keys()))
    frecs = np.asarray(list(dict_frec.values()))

    # create population
    sample_size = min(pop_size, len(possible_guesses))
    population = np.random.choice(a = possible_guesses, size = sample_size , replace = False, p = frecs)
    eligible_words = set()
    generation = 0

    # do genetic iterations
    while generation < max_gen:
        # selection
        fitnesses = [fitness(p, game_state) for p in population]
        selected = selection(fitnesses, population, k=tour_size)

        # new generation
        new_pop = []
        for p1, p2 in zip(selected[0::2], selected[1::2]):
            for c in crossover(p1, p2, prob=crossover_prob):
                c = mutate(c, prob=mutate_prob)
                c = permute(c, prob=permute_prob)
                c = invert(c, prob=invert_prob)

                if (c in eligible_words) or (c not in possible_guesses):
                    new_pop.append(np.random.choice(a = possible_guesses, p = frecs))
                else:
                    new_pop.append(c)

        population = new_pop
        eligible_words.update(population)

        generation += 1

    # choose word in eligible_words with maximum
    best_word = min(eligible_words, key=lambda x: fitness(x, game_state))
    dict_frec.pop(best_word)
    return best_word

In [158]:
def update_dict(dict_frec):
    frecs = np.asarray(list(dict_frec.values()))
    frecs = frecs/frecs.sum()
    return dict(zip(dict_frec.keys(), frecs))

In [159]:
def tryout(target, frecs):
    #Prep sets
    frec0 = frecs[0]
    dict_frec = frec0
    g0_sorted = sorted(frec0.items(), key=lambda x:x[1], reverse = True)
    g0_top = g0_sorted[:30]
    g0_top = np.asarray(g0_top)
    top_vals = g0_top[:,1].astype(float)
    top_vals = top_vals/top_vals.sum()
    top_labels = g0_top[:,0]
    game_state = []
    frecs = np.asarray(list(dict_frec.values()))
    frecs = (frecs + abs(min(frecs)))
    frecs = frecs/frecs.sum()
    dict_frec = dict(zip(dict_frec.keys(), frecs))
    word = wordle_genetic(game_state = game_state, guess_count = 0, pop_size = 150, 
                                         dict_frec = dict_frec, max_gen = 100, tour_size = 40,
                                        crossover_prob = 0.5,  mutate_prob = 0.03, permute_prob = 0.03, invert_prob = 0.02,
                                         top_vals = top_vals, top_labels = top_labels)
    check_words(word, target)
    game_state.append((word, check_words(word, target)))
    print(str(game_state[-1][1]) + " " +game_state[-1][0])
    i = 1
    while game_state[-1][1] != "🟩🟩🟩🟩🟩" and i <= 5:
        dict_frec = update_dict(dict_frec)
        word = wordle_genetic(game_state = game_state, guess_count = i, pop_size = 150, 
                                         dict_frec = dict_frec, max_gen = 100, tour_size = 40,
                                        crossover_prob = 0.5,  mutate_prob = 0.03, permute_prob = 0.03, invert_prob = 0.02,
                                         top_vals = top_vals, top_labels = top_labels)
        check_words(word, target)
        game_state.append((word, check_words(word, target)))
        print(str(game_state[-1][1]) + " " + game_state[-1][0])
        i += 1
    if game_state[-1][1] == "🟩🟩🟩🟩🟩":
        print("Ended with a win in " + str(i) + " attempts")
    else:
        print("Ended with a loss")

In [160]:
tryout('hello', frecs)

⬜🟨⬜🟩🟨 while
⬜⬜🟩🟨⬜ wyles
🟨🟨🟩⬜🟨 ollie
🟩🟩🟩🟩🟩 hello
Ended with a win in 4 attempts


In [162]:
tryout('where', frecs)

🟩🟩⬜⬜⬜ which
🟩🟩⬜⬜⬜ whoot
🟩🟩🟩⬜⬜ whens
🟩🟩🟩🟩🟩 where
Ended with a win in 4 attempts


In [177]:
tryout('allow', frecs)

⬜⬜🟨⬜⬜ state
⬜🟨⬜⬜⬜ copen
⬜⬜⬜🟩⬜ bhoot
🟩⬜⬜🟩🟩 arrow
🟩⬜🟩🟩🟩 aglow
🟩🟩🟩🟩🟩 allow
Ended with a win in 6 attempts


In [176]:
tryout('stair', frecs)

🟨⬜⬜⬜🟨 about
⬜⬜🟩🟨🟨 heats
⬜⬜🟩🟨⬜ ukase
⬜🟨🟨🟨⬜ waite
🟩🟩🟩🟩⬜ stain
🟩🟩🟩🟩⬜ staig
Ended with a loss


#### Improving mutation
We will create a dictionary with all the words that can transformed into another allowed one by only changing one letter

In [184]:
valid_inputs = []
with open("valid-wordle-words.txt") as f:
    valid_inputs = f.read().splitlines()
comparer = set(valid_inputs)
valid_mutations = {}
alphabet = list(map(chr, range(97, 123)))
for word in valid_inputs:
    valid_mutations[word] = set()
    for i in range(len(word)):
        for letter in alphabet:
            if (letter != word[i]) and ((word[:i] + letter + word[i+1:]) in comparer):
                valid_mutations[word].add(word[:i] + letter + word[i+1:])

In [218]:
def mutate2(word, dict_frec, prob=0.03):
    if random() < prob and len(valid_mutations[word]) > 0:
        frec = 0
        res = word
        for mut in valid_mutations[word]:
            if mut in dict_frec.keys() and dict_frec[mut] > frec:
                frec = dict_frec[mut]
                res = mut
        return res
    return word

#### Improving permutation
We will now create a dictionary with all the words that can be transformd into other valid words by swapping two non-equal letters.

In [198]:
valid_inputs = []
with open("valid-wordle-words.txt") as f:
    valid_inputs = f.read().splitlines()
comparer = set(valid_inputs)
valid_permutations = {}
changes = [[i,j] for i in range(5) for j in range(i+1,5)]
for word in valid_inputs:
    valid_permutations[word] = set()
    for i,j in changes:
        swap = list(word)
        swap[i], swap[j] = swap[j], swap[i]
        swap = ''.join(swap)
        if swap != word and swap in comparer:
            valid_permutations[word].add(swap)

In [219]:
def permute2(word, dict_frec, prob=0.03):
    if random() < prob and len(valid_permutations[word]) > 0:
        frec = 0
        res = word
        for mut in valid_permutations[word]:
            if mut in dict_frec and dict_frec[word] > frec:
                frec = dict_frec[word]
                res = mut
        return res
    return word

#### Improving inversion
We will create a dictionary of all possible inversions of a word.

In [211]:
valid_inputs = []
with open("valid-wordle-words.txt") as f:
    valid_inputs = f.read().splitlines()
comparer = set(valid_inputs)
valid_inversions = {}
inversions = [[i,j]for i in range(4) for j in range(i+1,5)]
for word in valid_inputs:
    valid_inversions[word] = set()
    for i,j in inversions:
        inv = list(word)
        inv[i:j+1] = reversed(word[i:j+1])
        inv = ''.join(inv)
        if inv != word and inv in comparer:
            valid_inversions[word].add(inv)

In [220]:
def invert2(word, dict_frec, prob=0.02):
    if random() < prob and len(valid_inversions[word]) > 0:
        frec = 0
        res = word
        for inv in valid_inversions[word]:
            if inv in dict_frec and dict_frec[word] > frec:
                frec = dict_frec[word]
                res = inv
        return res
    return word

In [246]:
def wordle_genetic(game_state, guess_count, pop_size, dict_frec, max_gen, tour_size, crossover_prob,
                   mutate_prob, permute_prob, invert_prob, top_vals, top_labels):
    
    if guess_count == 0:
        guess = first_word(top_labels, top_vals)
        dict_frec.pop(guess)
        return guess
    
    possible_guesses = np.asarray(list(dict_frec.keys()))
    frecs = np.asarray(list(dict_frec.values()))

    # create population
    sample_size = min(pop_size, len(possible_guesses))
    population = np.random.choice(a = possible_guesses, size = sample_size , replace = False, p = frecs)
    eligible_words = set()
    generation = 0

    # do genetic iterations
    while generation < max_gen:
        # selection
        fitnesses = [fitness(p, game_state) for p in population]
        selected = selection(fitnesses, population, k=tour_size)

        # new generation
        new_pop = []
        for p1, p2 in zip(selected[0::2], selected[1::2]):
            for c in crossover(p1, p2, prob=crossover_prob):
                if (c in eligible_words) or (c not in possible_guesses):
                    new_pop.append(np.random.choice(a = possible_guesses, p = frecs))
                else:
                    c = mutate2(c, dict_frec, prob=mutate_prob)
                    c = permute2(c, dict_frec, prob=permute_prob)
                    c = invert2(c, dict_frec, prob=invert_prob)
                    new_pop.append(c)

        population = new_pop
        eligible_words.update(population)

        generation += 1

    # choose word in eligible_words with maximum
    best_word = min(eligible_words, key=lambda x: fitness(x, game_state))
    dict_frec.pop(best_word)
    return best_word

In [258]:
tryout('humor', frecs)

⬜🟨🟨⬜⬜ could
⬜🟩🟩⬜⬜ dummy
⬜🟩⬜⬜⬜ lusty
⬜🟩🟨⬜⬜ durra
🟩🟩🟩🟩🟩 humor
Ended with a win in 5 attempts
